
# manipulate files containing storm tracks from TE
# this script is used to retain only time steps at noon in the *txt files resulting from TempestExtremes
# this script is used to add the time in the *txt files resulting from TempestExtremes


In [1]:

import sys
sys.path.append('/global/homes/c/cnissen/scripts/seawater-3.3.4/seawater/')
import os
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import cm
import seawater
#from seawater import dist
#import seawater as sw
import matplotlib.path as mpath
from cartopy.util import add_cyclic_point
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import random
from numba import njit
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
import copy
from datetime import date, datetime, timedelta

In [2]:
#----
# load storm tracks from tempest extremes
#----

pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
region_string = '4080S'
fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'.txt' # load original file here!
print(fileTE)

#---
# read all storm positions for year1
#---

year_storm  = []
month_storm = []
day_storm   = []
hour_storm  = []
index_lon   = []
index_lat   = []
lon_storm   = []
lat_storm   = []
slp_center  = []
wind_max    = []
count_storms = []
duration_storm = []
counter = 0
with open(pathTE+fileTE, 'r') as f:
    for line in tqdm(f):
        aux = line.split()
        if aux[0]!='start': #year1: # starting a new time entry
            # 262     77      147.375000      -46.333780      1.003353e+05    1.493876e+01    2012    1       2       21
            index_lon   += [int(aux[0])]
            index_lat   += [int(aux[1])]
            lon_storm   += [float(aux[2])]
            lat_storm   += [float(aux[3])]
            year_storm  += [int(aux[6])]
            month_storm += [int(aux[7])]
            day_storm   += [int(aux[8])]
            hour_storm  += [int(aux[9])]
            slp_center  += [float(aux[4])]
            wind_max    += [float(aux[5])]
            count_storms+= [counter]
        else:
            # start   29      2012    1       15      21
            duration_storm+= [int(aux[1])]
            counter=counter+1
        del aux
        
print(len(lon_storm),len(year_storm),len(month_storm),len(slp_center))
print('Number of storms:',np.max(count_storms))
print('avg. duration of storms:',np.mean(duration_storm))
print ('done')


SH_stormtraj_1997_2018_4080S.txt


363754it [00:00, 365964.99it/s]


354136 354136 354136 354136
Number of storms: 9618
avg. duration of storms: 36.82012892493242
done


In [3]:
#---
# get minimum slp for each storm 
# get month of minimum for each storm
# note: only needed if storms are filtered based on these (script provides options to do that; paper doesn't end up using these though)
#---

ind = 0
slp_min       = np.zeros(len(duration_storm))
wind_max_storm= np.zeros(len(duration_storm))
wind_max_storm_when= np.zeros(len(duration_storm))
month_slp_min = np.zeros(len(duration_storm))
lat_slp_min   = np.zeros(len(duration_storm))
lon_slp_min   = np.zeros(len(duration_storm))
for ss in tqdm(range(0,len(duration_storm))):
    # duration_storm[ss] = how many entries for storm?
    #print(ind,duration_storm[ss])
    slp_min[ss] = np.min(slp_center[ind:(ind+duration_storm[ss])])
    wind_max_storm[ss] = np.max(wind_max[ind:(ind+duration_storm[ss])])
    wind_max_storm_when[ss] = np.argmax(wind_max[ind:(ind+duration_storm[ss])])
    # month 
    ind_tt = np.argmin(slp_center[ind:(ind+duration_storm[ss])])
    month_slp_min[ss] = month_storm[ind:(ind+duration_storm[ss])][ind_tt]
    del ind_tt
    # lat 
    ind_tt = np.argmin(slp_center[ind:(ind+duration_storm[ss])])
    lat_slp_min[ss] = lat_storm[ind:(ind+duration_storm[ss])][ind_tt]
    del ind_tt
    # lon 
    ind_tt = np.argmin(slp_center[ind:(ind+duration_storm[ss])])
    lon_slp_min[ss] = lon_storm[ind:(ind+duration_storm[ss])][ind_tt]
    del ind_tt
    ind = ind+duration_storm[ss]

print(len(month_slp_min))


100%|██████████| 9618/9618 [00:00<00:00, 29501.95it/s]


In [ ]:
#---
# reduce storm track to all time steps at noon
#---
# the resulting file is the basis for the analysis in the paper (CESM output is daily averaged, so only look at storm positions at noon)

create_at_noon = False
if create_at_noon:
    # step 1a:
    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
    region_string = '4080S'
    fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'.txt'

    # create a new file
    fileTE_aux = 'aux.txt'
    fileTE_new = fileTE[:-4]+'_at_noon.txt'
    print(fileTE_new)
    
    slp_min_use = np.copy(slp_min) # all storms (no matter the season)

    # loop over all storms and check whether it is at noon
    counter = 0
    count_storms = [] # count storms for current season
    year_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    month_peak = np.zeros(len(slp_min_use)) # info at storm peak
    day_peak   = np.zeros(len(slp_min_use)) # info at storm peak
    hour_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_new, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                count_storms+= [counter]
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                counter=counter+1
            else: 
                #----
                # save storm info in txt file
                #for i in aux:
                #print(aux[9],aux)
                if int(aux[9])==12: # store location at noon
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            f_out.write('\t')
                            f_out.write(aux[i]) #+'\t')
                        else: # last column
                            f_out.write('\t')
                            f_out.write(aux[i])
                            f_out.write('\t')
                            f_out.write(str(count_storms[-1]+1)) # add index for current storm as a column 
                            f_out.write('\n')
                #----
            del aux


    print(len(count_storms))

    

In [6]:
#----
# manipulate storm tracks: shift date by XX days
# --> here: take *at_noon* file as starting point (see above)
# --> hypothesis: maximum chl response occurs after the storm has passed
#----
# variable "shift_days" determines by how many days the entries are shifted (plus or minus!)
#
# NOTE: I don't manipulate the start date of the storm here! (the "at_noon" file also did not do that...)

create_at_noon_shift_time = False
if create_at_noon_shift_time:
    # step 1a:
    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
    region_string = '4080S'
    fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'.txt'
    
    # FUNCTION to shift date (from here: https://www.30secondsofcode.org/python/s/add-days/)
    def add_days(n, d = datetime.today()):
        return d + timedelta(n)

    shift_days = 5 # can also be negative!
        
    # create a new file
    fileTE_aux = 'aux.txt'
    if shift_days>0:
        fileTE_new = fileTE[:-4]+'_at_noon_plus_'+str(shift_days)+'_days.txt'
    else:
        fileTE_new = fileTE[:-4]+'_at_noon_minus_'+str(-1*shift_days)+'_days.txt'
    print(fileTE_new)
    
    slp_min_use = np.copy(slp_min) # all storms (no matter the season)

    # loop over all storms and check whether it is at noon
    counter = 0
    count_storms = [] # count storms for current season
    year_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    month_peak = np.zeros(len(slp_min_use)) # info at storm peak
    day_peak   = np.zeros(len(slp_min_use)) # info at storm peak
    hour_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_new, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                count_storms+= [counter]
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                counter=counter+1
            else: 
                #----
                # save storm info in txt file
                #----
                #-----
                # manipulate start date of the storm
                aux_day   = aux[8] # from 1-last_day_of_month
                aux_month = aux[7] # from 1-12
                aux_year  = aux[6]
                aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                #----                    
                if int(aux[9])==12: # store location at noon
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==6: # year
                                f_out.write('\t')
                                f_out.write(str(aux_date.year))
                            elif i==7: # month
                                f_out.write('\t')
                                f_out.write(str(aux_date.month))
                            elif i==8: # day
                                f_out.write('\t')
                                f_out.write(str(aux_date.day))
                            else:
                                f_out.write('\t')
                                f_out.write(aux[i]) #+'\t')
                        else: # last column
                            f_out.write('\t')
                            f_out.write(aux[i])
                            f_out.write('\t')
                            f_out.write(str(count_storms[-1]+1)) # add index for current storm as a column 
                            f_out.write('\n')
                #----
            del aux

    print(len(count_storms))


SH_stormtraj_1997_2018_4080S_at_noon_plus_5_days.txt


363754it [00:00, 410980.18it/s]

9618


In [ ]:
#----
# EVERYTHING BELOW THIS CELL WAS NOT USED IN PAPER (I initially tested a few things)
#----

In [ ]:
#----
# reduce storm track to all time steps at noon
# manipulate storm tracks for each season: shift date by XX days
# --> hypothesis: maximum chl response occurs after the storm has passed
#----

create_at_noon_shift_time_seasons = False
if create_at_noon_shift_time_seasons:

    print(month_slp_min.shape)
    print(np.min(month_slp_min),np.max(month_slp_min))
    print(len(duration_storm))

    #pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    #fileTE = 'SH_stormtraj_1958_2019_4080S.txt'
    
    print('original fileTE:',fileTE)

    # create a new file
    season = 'DJF'
    shift_days = 1 # can also be negative!
    if shift_days>0:
        fileTE_season = fileTE[:-4]+'_'+season+'_plus_'+str(shift_days)+'_days.txt'
    else:
        fileTE_season = fileTE[:-4]+'_'+season+'_minus_'+str(-1*shift_days)+'_days.txt'
    print(fileTE_season)

    # get all storms for which SLP minimum is in current season
    if season in ['DJF']:
        ind_mo = np.where((month_slp_min==12) | (month_slp_min==1) | (month_slp_min==2))[0]
    elif season in ['MAM']:
        ind_mo = np.where((month_slp_min==3) | (month_slp_min==4) | (month_slp_min==5))[0]
    elif season in ['JJA']:
        ind_mo = np.where((month_slp_min==6) | (month_slp_min==7) | (month_slp_min==8))[0]
    elif season in ['SON']:
        ind_mo = np.where((month_slp_min==9) | (month_slp_min==10) | (month_slp_min==11))[0]
    elif season in ['Oct']:
        ind_mo = np.where((month_slp_min==10))[0]
    elif season in ['Mar']:
        ind_mo = np.where((month_slp_min==3))[0]
    print('Number of storms:',ind_mo.shape[0]) # np.min(ind_mo) -> note that index starts at 0 here!

    # how many days in each month? (needed to determine whether the month entry also needs to be updated!)
    #num_days = [31,28,31,30,31,30,31,31,30,31,30,31]

    # FUNCTION to shift date (from here: https://www.30secondsofcode.org/python/s/add-days/)
    def add_days(n, d = datetime.today()):
        return d + timedelta(n)

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    #month_storm_season = [] 
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_season, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                save_storm=False
                # check if storm is in the season I am trying to filter
                if counter in ind_mo: 
                    count_storms+= [counter]
                    save_storm=True       
                    #-----
                    # manipulate start date of the storm
                    aux_day   = aux[4]   # from 1-last_day_of_month
                    aux_month = aux[3] # from 1-12
                    aux_year  = aux[2]
                    aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                    #----
                    # save first line of storm info in txt file
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==2: # year
                                f_out.write(str(aux_date.year)+'\t')
                            elif i==3: # month
                                f_out.write(str(aux_date.month)+'\t')
                            elif i==4: # day
                                f_out.write(str(aux_date.day)+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
                    del aux_date,aux_day,aux_month,aux_year
                counter=counter+1
            else: 
                if save_storm: # store information of current storm 
                    #-----
                    # manipulate start date of the storm
                    aux_day   = aux[8]   # from 1-last_day_of_month
                    aux_month = aux[7] # from 1-12
                    aux_year  = aux[6]
                    aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                    #----
                    #----
                    # save storm info in txt file
                    f_out.write('\t')
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==6: # year
                                f_out.write(str(aux_date.year)+'\t')
                            elif i==7: # month
                                f_out.write(str(aux_date.month)+'\t')
                            elif i==8: # day
                                f_out.write(str(aux_date.day)+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
            del aux

    #print(len(count_storms))


In [ ]:
#---
# reduce storm track to the time of the min. SLP
#---

create_at_peak = False
if create_at_peak:
    # step 1a:
    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
    region_string = '4080S'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95710Pa.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94810Pa.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95710Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94810Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95760Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94830Pa_4ts.txt'
    
    # SH_stormtraj_2009_2015_4080S_winds_above_25_8ts.txt

    # create a new file
    fileTE_aux = 'aux.txt'
    fileTE_new = fileTE[:-4]+'_at_min_SLP.txt'
    print(fileTE_new)
    
    slp_min_use = np.copy(slp_min) # all storms (no matter the season)

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    year_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    month_peak = np.zeros(len(slp_min_use)) # info at storm peak
    day_peak   = np.zeros(len(slp_min_use)) # info at storm peak
    hour_peak  = np.zeros(len(slp_min_use)) # info at storm peak
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_aux, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                count_storms+= [counter]
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                counter=counter+1
            else: 
                #----
                # save storm info in txt file
                #for i in aux:
                for i in range(0,len(aux)):
                    if float(aux[4])==slp_min_use[counter-1]: # once the min. SLP is reached, write this line but then stop writing
                        if i<(len(aux)-1):
                            f_out.write('\t')
                            f_out.write(aux[i]) #+'\t')
                        else:
                            f_out.write('\t')
                            f_out.write(aux[i])
                            f_out.write('\n')
                        if i==6:
                            year_peak[counter-1]  = int(aux[6])
                        elif i==7: 
                            month_peak[counter-1] = int(aux[7])
                        elif i==8: 
                            day_peak[counter-1]   = int(aux[8])
                        elif i==9: 
                            hour_peak[counter-1]  = int(aux[9])
                        
                #----
            del aux
            
    # TO DO: correct number of JRA time steps and the date given in the first line of each storm
    # for each "start" line in file, replace the second entry with the updated number
    print('Correct duration and date of storm peak in file...')
    counter = 0
    with open(pathTE+fileTE_aux, 'r') as f, open(pathTE+fileTE_new, 'w') as f_out:  
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start':
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        if i==1: # second entry: duration of storm in file (only peak is stored -> set to 1 here)
                            f_out.write(str(1)+'\t')
                        elif i==2:
                            f_out.write(str(int(year_peak[counter]))+'\t')
                        elif i==3:
                            f_out.write(str(int(month_peak[counter]))+'\t')
                        elif i==4:
                            f_out.write(str(int(day_peak[counter]))+'\t')
                        elif i==5:
                            f_out.write(str(int(hour_peak[counter]))+'\t')
                        else:
                            f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(str(int(hour_peak[counter])))
                        #f_out.write(aux[i])     
                f_out.write('\n')
                #----
                counter=counter+1
            else:
                for i in range(0,len(aux)):
                    if float(aux[4])==slp_min_use[counter-1]: # once the min. SLP is reached, write this line but then stop writing
                        if i<(len(aux)-1):
                            f_out.write('\t')
                            f_out.write(aux[i]) #+'\t')
                        else:
                            f_out.write('\t')
                            f_out.write(aux[i])
                            f_out.write('\n')                
            del aux


    print(len(count_storms))


In [ ]:
#---
# MANIPULATE STORM TRACKS; reduce storm track to the time of the min. SLP
#---
# --> hypothesis: maximum chl response occurs after the storm has passed
#----

create_at_peak_shifted_time = True
if create_at_peak_shifted_time:
    # step 1a:
    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    time_string   = '1997_2018'  # '1979_2018' '2009_2018', 1979_1988, 2009_2015
    region_string = '4080S'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'.txt'
    fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95710Pa.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94810Pa.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95710Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94810Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_95760Pa_4ts.txt'
    #fileTE = 'SH_stormtraj_'+time_string+'_'+region_string+'_slp_below_94830Pa_4ts.txt'

    # create a new file
    fileTE_aux = 'aux.txt'
    fileTE_new = fileTE[:-4]+'_at_min_SLP.txt'
    print('original fileTE:',fileTE_new)

    # create a new file
    shift_days = 1 # can also be negative!
    if shift_days>0:
        fileTE_shifted = fileTE_new[:-4]+'_plus_'+str(shift_days)+'_days.txt'
    else:
        fileTE_shifted = fileTE_new[:-4]+'_minus_'+str(-1*shift_days)+'_days.txt'
    print(fileTE_shifted)
    
    # FUNCTION to shift date (from here: https://www.30secondsofcode.org/python/s/add-days/)
    def add_days(n, d = datetime.today()):
        return d + timedelta(n)

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    #month_storm_season = [] 
    with open(pathTE+fileTE_new, 'r') as f, open(pathTE+fileTE_shifted, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                # check if storm is in the season I am trying to filter
                count_storms+= [counter]
                #-----
                # manipulate start date of the storm
                aux_day   = aux[4]   # from 1-last_day_of_month
                aux_month = aux[3] # from 1-12
                aux_year  = aux[2]
                aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        if i==2: # year
                            f_out.write(str(aux_date.year)+'\t')
                        elif i==3: # month
                            f_out.write(str(aux_date.month)+'\t')
                        elif i==4: # day
                            f_out.write(str(aux_date.day)+'\t')
                        else:
                            f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                del aux_date,aux_day,aux_month,aux_year
                counter=counter+1
            else: 
                #-----
                # manipulate start date of the storm
                aux_day   = aux[8]   # from 1-last_day_of_month
                aux_month = aux[7] # from 1-12
                aux_year  = aux[6]
                aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                #----
                # save storm info in txt file
                f_out.write('\t')
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        if i==6: # year
                            f_out.write(str(aux_date.year)+'\t')
                        elif i==7: # month
                            f_out.write(str(aux_date.month)+'\t')
                        elif i==8: # day
                            f_out.write(str(aux_date.day)+'\t')
                        else:
                            f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
            del aux

    print(len(count_storms))



In [ ]:
#----
# create files with storm tracks for each season
#----

create_seasonal = False
if create_seasonal:
    print(month_slp_min.shape)
    print(np.min(month_slp_min),np.max(month_slp_min))
    print(len(duration_storm))

    # create a new file
    season = 'SON'
    fileTE_season = fileTE[:-4]+'_'+season+'.txt'
    print(fileTE_season)

    # get all storms for which SLP minimum is in current season
    if season in ['DJF']:
        ind_mo = np.where((month_slp_min==12) | (month_slp_min==1) | (month_slp_min==2))[0]
    elif season in ['MAM']:
        ind_mo = np.where((month_slp_min==3) | (month_slp_min==4) | (month_slp_min==5))[0]
    elif season in ['JJA']:
        ind_mo = np.where((month_slp_min==6) | (month_slp_min==7) | (month_slp_min==8))[0]
    elif season in ['SON']:
        ind_mo = np.where((month_slp_min==9) | (month_slp_min==10) | (month_slp_min==11))[0]
    elif season in ['Oct']:
        ind_mo = np.where((month_slp_min==10))[0]
    elif season in ['Mar']:
        ind_mo = np.where((month_slp_min==3))[0]

    print('Number of storms:',ind_mo.shape[0]) # np.min(ind_mo) -> note that index starts at 0 here!

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    #month_storm_season = [] 
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_season, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                save_storm=False
                # check if storm is in the season I am trying to filter
                if counter in ind_mo: 
                    count_storms+= [counter]
                    save_storm=True         
                    #----
                    # save first line of storm info in txt file
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
                counter=counter+1
            else: 
                if save_storm: # store information of current storm 
                    #month_storm_season += [int(aux[7])]
                    #----
                    # save storm info in txt file
                    f_out.write('\t')
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
            del aux

    #print(len(count_storms))


In [ ]:
#----
# storms in each season: create files with separate storm tracks up to min. SLP
#----

create_pre_peak = False
if create_pre_peak:

    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    #fileTE = 'SH_stormtraj_1958_2019_4080S.txt'

    # create a new file
    season = 'DJF'
    fileTE_season = fileTE[:-4]+'_'+season+'.txt'
    fileTE_season_reduced = 'aux_file.txt' # has the wrong number in header for the duration of each storm
    fileTE_season_reduced_final = fileTE[:-4]+'_'+season+'_pre_peak.txt' # with corrected number for duration of storm
    print(fileTE_season_reduced)
    print(fileTE_season_reduced_final)

    # get all storms for which SLP minimum is in current season
    if season in ['DJF']:
        ind_mo = np.where((month_slp_min==12) | (month_slp_min==1) | (month_slp_min==2))[0]
    elif season in ['MAM']:
        ind_mo = np.where((month_slp_min==3) | (month_slp_min==4) | (month_slp_min==5))[0]
    elif season in ['JJA']:
        ind_mo = np.where((month_slp_min==6) | (month_slp_min==7) | (month_slp_min==8))[0]
    elif season in ['SON']:
        ind_mo = np.where((month_slp_min==9) | (month_slp_min==10) | (month_slp_min==11))[0]
    slp_min_use = slp_min[ind_mo] # reduce to storms contained in seasonal file

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    duration_storm_pre_peak = np.zeros(len(slp_min_use)) # how many JRA time steps in each storm?
    with open(pathTE+fileTE_season, 'r') as f, open(pathTE+fileTE_season_reduced, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                count_storms+= [counter]
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                counter=counter+1
                to_write = True
            else: 
                #----
                # save storm info in txt file
                if to_write:
                    f_out.write('\t')
                #for i in aux:
                for i in range(0,len(aux)):
                    if float(aux[4])==slp_min_use[counter-1]: # once the min. SLP is reached, write this line but then stop writing
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                        to_write = False # stop writing remaining lines of curent storm
                        written = True # to control whether or not a new line is started
                    else:
                        if to_write:
                            if i<(len(aux)-1):
                                f_out.write(aux[i]+'\t')
                            else:
                                f_out.write(aux[i])
                            written = True # to control whether or not a new line is started
                        else:
                            written=False # to control whether or not a new line is started
                if written:
                    duration_storm_pre_peak[counter-1] = duration_storm_pre_peak[counter-1]+1
                    f_out.write('\n')
                #----
            del aux

    print(len(count_storms))

    # TO DO: correct number of JRA time steps given in the first line of each storm
    # I stored updated duration of the storm in "duration_storm_pre_peak"
    # for each "start" line in file, replace the second entry with the updated number
    print('Corrected duration of storm in file...')
    counter = 0
    with open(pathTE+fileTE_season_reduced, 'r') as f, open(pathTE+fileTE_season_reduced_final, 'w') as f_out:  
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start':
                #print(len(aux),aux)
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        if i==1: # second entry
                            f_out.write(str(int(duration_storm_pre_peak[counter]))+'\t')
                        else:
                            f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])     
                f_out.write('\n')
                #----
                counter=counter+1
                to_write = True
            else: 
                #----
                # save storm info in txt file
                if to_write:
                    f_out.write('\t')
                for i in range(0,len(aux)):
                    if float(aux[4])==slp_min_use[counter-1]: # once the min. SLP is reached, write this line but then stop writing
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                        to_write = False # stop writing remaining lines of curent storm
                        written = True # to control whether or not a new line is started
                    else:
                        if to_write:
                            if i<(len(aux)-1):
                                f_out.write(aux[i]+'\t')
                            else:
                                f_out.write(aux[i])
                            written = True # to control whether or not a new line is started
                        else:
                            written=False # to control whether or not a new line is started
                if written:
                    f_out.write('\n')
                #----
            del aux




In [ ]:
#----
# manipulate storm tracks for each season: shift date by XX days
# --> hypothesis: maximum chl response occurs after the storm has passed
#----

create_shift_time = False
if create_shift_time:

    print(month_slp_min.shape)
    print(np.min(month_slp_min),np.max(month_slp_min))
    print(len(duration_storm))

    #pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    #fileTE = 'SH_stormtraj_1958_2019_4080S.txt'
    
    print('original fileTE:',fileTE)

    # create a new file
    season = 'DJF'
    shift_days = 1 # can also be negative!
    if shift_days>0:
        fileTE_season = fileTE[:-4]+'_'+season+'_plus_'+str(shift_days)+'_days.txt'
    else:
        fileTE_season = fileTE[:-4]+'_'+season+'_minus_'+str(-1*shift_days)+'_days.txt'
    print(fileTE_season)

    # get all storms for which SLP minimum is in current season
    if season in ['DJF']:
        ind_mo = np.where((month_slp_min==12) | (month_slp_min==1) | (month_slp_min==2))[0]
    elif season in ['MAM']:
        ind_mo = np.where((month_slp_min==3) | (month_slp_min==4) | (month_slp_min==5))[0]
    elif season in ['JJA']:
        ind_mo = np.where((month_slp_min==6) | (month_slp_min==7) | (month_slp_min==8))[0]
    elif season in ['SON']:
        ind_mo = np.where((month_slp_min==9) | (month_slp_min==10) | (month_slp_min==11))[0]
    elif season in ['Oct']:
        ind_mo = np.where((month_slp_min==10))[0]
    elif season in ['Mar']:
        ind_mo = np.where((month_slp_min==3))[0]
    print('Number of storms:',ind_mo.shape[0]) # np.min(ind_mo) -> note that index starts at 0 here!

    # how many days in each month? (needed to determine whether the month entry also needs to be updated!)
    #num_days = [31,28,31,30,31,30,31,31,30,31,30,31]

    # FUNCTION to shift date (from here: https://www.30secondsofcode.org/python/s/add-days/)
    def add_days(n, d = datetime.today()):
        return d + timedelta(n)

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    #month_storm_season = [] 
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_season, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                save_storm=False
                # check if storm is in the season I am trying to filter
                if counter in ind_mo: 
                    count_storms+= [counter]
                    save_storm=True       
                    #-----
                    # manipulate start date of the storm
                    aux_day   = aux[4]   # from 1-last_day_of_month
                    aux_month = aux[3] # from 1-12
                    aux_year  = aux[2]
                    aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                    #----
                    # save first line of storm info in txt file
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==2: # year
                                f_out.write(str(aux_date.year)+'\t')
                            elif i==3: # month
                                f_out.write(str(aux_date.month)+'\t')
                            elif i==4: # day
                                f_out.write(str(aux_date.day)+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
                    del aux_date,aux_day,aux_month,aux_year
                counter=counter+1
            else: 
                if save_storm: # store information of current storm 
                    #-----
                    # manipulate start date of the storm
                    aux_day   = aux[8]   # from 1-last_day_of_month
                    aux_month = aux[7] # from 1-12
                    aux_year  = aux[6]
                    aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                    #----
                    #----
                    # save storm info in txt file
                    f_out.write('\t')
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==6: # year
                                f_out.write(str(aux_date.year)+'\t')
                            elif i==7: # month
                                f_out.write(str(aux_date.month)+'\t')
                            elif i==8: # day
                                f_out.write(str(aux_date.day)+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
            del aux

    #print(len(count_storms))


In [ ]:
#---
# reduce storm track to XX time steps before & after the min. SLP
#---

create_close_to_peak = False
if create_close_to_peak:
    # step 1a:
    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    fileTE = 'SH_stormtraj_1958_2019_4080S_winds_above_30_8ts.txt'

    # create a new file
    season = 'Mar'
    ts_around_min_slp = 8 # 8 = 1day, 12 = 1.5 days
    fileTE_season = fileTE[:-4]+'_'+season+'.txt'
    fileTE_season_reduced = 'aux_file.txt' # has the wrong number in header for the duration of each storm
    fileTE_season_reduced_final = fileTE[:-4]+'_'+season+'_'+str(ts_around_min_slp)+'_ts_around_min_SLP.txt' # with corrected number for duration of storm
    print(fileTE_season_reduced)
    print(fileTE_season_reduced_final)

    # get all storms for which SLP minimum is in current season
    if season in ['DJF']:
        ind_mo = np.where((month_slp_min==12) | (month_slp_min==1) | (month_slp_min==2))[0]
    elif season in ['MAM']:
        ind_mo = np.where((month_slp_min==3) | (month_slp_min==4) | (month_slp_min==5))[0]
    elif season in ['JJA']:
        ind_mo = np.where((month_slp_min==6) | (month_slp_min==7) | (month_slp_min==8))[0]
    elif season in ['SON']:
        ind_mo = np.where((month_slp_min==9) | (month_slp_min==10) | (month_slp_min==11))[0]
    elif season in ['Oct']:
        ind_mo = np.where((month_slp_min==10))[0]
    elif season in ['Mar']:
        ind_mo = np.where((month_slp_min==3))[0]
    slp_min_use             = slp_min[ind_mo] # reduce to storms contained in seasonal file
    wind_max_storm_use      = wind_max_storm[ind_mo] # reduce to storms contained in seasonal file 
    wind_max_storm_when_use = wind_max_storm_when[ind_mo]
    duration_storm_use      = np.asarray(duration_storm)[ind_mo]

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    duration_storm_pre_peak = np.zeros(len(slp_min_use)) # how many JRA time steps in each storm?
    num_lines_storm = np.zeros([len(slp_min_use)])
    write_storm     = np.zeros([len(slp_min_use)]) # keep track of which storms have (not) been written to file
    # for each storm, record first time step in file --> needed to correct "start" line in step 1b (see below)
    year_first  = -999*np.ones([len(slp_min_use)])
    month_first = -999*np.ones([len(slp_min_use)])
    day_first   = -999*np.ones([len(slp_min_use)])
    hour_first  = -999*np.ones([len(slp_min_use)])
    #1958    1       5       0
    with open(pathTE+fileTE_season, 'r') as f, open(pathTE+fileTE_season_reduced, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                count_storms+= [counter]

                # construct a check for what lines should be written to file
                # --> depends on number of time steps before/after peak I want in file and whether these are actually available
                # --> if they are not, do not output this storm
                write_lines = np.arange((wind_max_storm_when_use[counter]-ts_around_min_slp),\
                                        (wind_max_storm_when_use[counter]+ts_around_min_slp)+1,1)
                # TEST: subtract 1
                write_lines = [x-1 for x in write_lines]
                write_lines = np.asarray(write_lines)
                # only include available time steps before wind maximum
                write_lines = write_lines[write_lines>=0] 
                # only include available time steps after wind maximum
                write_lines = write_lines[write_lines<=(duration_storm_use[counter]-1)] 
                # convert to integers
                write_lines = [int(x) for x in write_lines]

                if len(write_lines)==((2*ts_around_min_slp)+1):
                    if counter>0:
                        print('# of lines written:',count_written) # of previous storm
                    print(len(write_lines),wind_max_storm_when_use[counter],write_lines,aux)
                    to_write = True
                    write_storm[counter] = 1
                    num_lines_storm[counter] = len(write_lines) # only store info for storms that I write to file 
                    #  (all other entries are zero --> important for correction in step 1b)
                    count_written = 0 # initalze counter for lines written in current storm
                else:
                    to_write = False
                #----
                # save first line of storm info in txt file
                if to_write:
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                #----
                if int(aux[2])==1989:
                    to_print=True
                else:
                    to_print=False
                counter=counter+1
                counter_ts_storm = 0 # initialze counter for current storm
                #print(write_lines)
            else: 
                #----
                # save storm info in txt file
                if to_write:
                    if counter_ts_storm in write_lines: # write line if time step is of interest
                        if to_print:
                            print(counter_ts_storm,aux)
                        if year_first[counter-1]==-999: # only write the first entry of each storm!
                            #print(aux)
                            year_first[counter-1]  = int(aux[6])
                            month_first[counter-1] = int(aux[7])
                            day_first[counter-1]   = int(aux[8])
                            hour_first[counter-1]  = int(aux[9])
                        for i in range(0,len(aux)): # loop over columns in current line
                            if i==0: 
                                f_out.write('\t')
                            if i<(len(aux)-1):
                                f_out.write(aux[i]+'\t')
                            else:
                                f_out.write(aux[i])
                                f_out.write('\n')
                        count_written = count_written+1
                    counter_ts_storm = counter_ts_storm+1
            del aux


    # STEP 1b: correct number of JRA time steps given in the first line of each storm
    # I stored updated duration of the storm in "duration_storm_pre_peak"
    # for each "start" line in file, replace the second entry with the updated number
    print('Corrected duration of storm in file...')
    counter = 0
    # correct time info in "start" line ("counter" in this loop already only includes the ones contained in file!)
    year_first  = year_first[year_first>=0]
    month_first = month_first[month_first>=0]
    day_first   = day_first[day_first>=0]
    hour_first  = hour_first[hour_first>=0]
    num_lines_storm  = num_lines_storm[num_lines_storm>0]
    with open(pathTE+fileTE_season_reduced, 'r') as f, open(pathTE+fileTE_season_reduced_final, 'w') as f_out:  
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start':
                #print(len(aux),aux)
                #----
                # save first line of storm info in txt file
                to_write = False
                to_process = True
                if to_process: #write_storm[counter]==1: # if storm is in file
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==1: # second entry
                                f_out.write(str(int(num_lines_storm[counter]))+'\t')
                            elif i==2: 
                                f_out.write(str(int(year_first[counter]))+'\t')
                            elif i==3: 
                                f_out.write(str(int(month_first[counter]))+'\t')
                            elif i==4: 
                                f_out.write(str(int(day_first[counter]))+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(str(int(hour_first[counter])))
                            #f_out.write(aux[i])     
                    f_out.write('\n')
                    #----
                    to_write = True
                   # if counter<10:
                   #     print(counter,year_first[counter],month_first[counter],day_first[counter],hour_first[counter])
                counter=counter+1            
            else: 
                #----
                # save storm info in txt file
                if to_write:
                    for i in range(0,len(aux)):
                        if i==0:
                            f_out.write('\t')
                           # if counter<10:
                           #     print(aux)
                        if i<(len(aux)-1):
                            f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                            f_out.write('\n')
                #----
            del aux


    print(len(count_storms))


In [ ]:
#-----
# shift time of storm tracks that were reduced to XX time steps/days around peak intensity
# -> composite chl along track when the storm was most intense
#-----
# first test: take single storm examples

create_shift_time_close_to_peak = False
if create_shift_time_close_to_peak:
    print(month_slp_min.shape)
    print(np.min(month_slp_min),np.max(month_slp_min))
    print(len(duration_storm))

    pathTE = '/global/cfs/cdirs/m4003/cnissen/TempestExtremes_output/'
    #fileTE = 'SH_stormtraj_1958_2019_4080S_winds_above_30_8ts_Mar_8_ts_around_min_SLP_single_storm.txt'
    fileTE = 'SH_stormtraj_1958_2019_4080S_winds_above_30_8ts_Oct_8_ts_around_min_SLP_single_storm.txt'

    # create a new file
    shift_days = 1 # can also be negative!
    if shift_days>0:
        fileTE_season = fileTE[:-4]+'_plus_'+str(shift_days)+'_days.txt'
    else:
        fileTE_season = fileTE[:-4]+'_minus_'+str(-1*shift_days)+'_days.txt'
    print(fileTE_season)

    # how many days in each month? (needed to determine whether the month entry also needs to be updated!)
    #num_days = [31,28,31,30,31,30,31,31,30,31,30,31]

    # FUNCTION to shift date (from here: https://www.30secondsofcode.org/python/s/add-days/)
    def add_days(n, d = datetime.today()):
        return d + timedelta(n)

    # loop over all storms and check whether it is in the season 
    counter = 0
    count_storms = [] # count storms for current season
    #month_storm_season = [] 
    with open(pathTE+fileTE, 'r') as f, open(pathTE+fileTE_season, 'w') as f_out:  # go through original file
        for line in tqdm(f):
            aux = line.split()
            if aux[0]=='start': # new storm
                save_storm=False
                # check if storm is in the season I am trying to filter

                count_storms+= [counter]
                save_storm=True       
                #-----
                # manipulate start date of the storm
                aux_day   = aux[4]   # from 1-last_day_of_month
                aux_month = aux[3] # from 1-12
                aux_year  = aux[2]
                aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                #----
                # save first line of storm info in txt file
                for i in range(0,len(aux)):
                    if i<(len(aux)-1):
                        if i==2: # year
                            f_out.write(str(aux_date.year)+'\t')
                        elif i==3: # month
                            f_out.write(str(aux_date.month)+'\t')
                        elif i==4: # day
                            f_out.write(str(aux_date.day)+'\t')
                        else:
                            f_out.write(aux[i]+'\t')
                    else:
                        f_out.write(aux[i])
                f_out.write('\n')
                #----
                del aux_date,aux_day,aux_month,aux_year
                counter=counter+1
            else: 
                if save_storm: # store information of current storm 
                    #-----
                    # manipulate start date of the storm
                    aux_day   = aux[8]   # from 1-last_day_of_month
                    aux_month = aux[7] # from 1-12
                    aux_year  = aux[6]
                    aux_date  = add_days(shift_days, date(int(aux_year), int(aux_month), int(aux_day)))                          
                    #----
                    #----
                    # save storm info in txt file
                    f_out.write('\t')
                    for i in range(0,len(aux)):
                        if i<(len(aux)-1):
                            if i==6: # year
                                f_out.write(str(aux_date.year)+'\t')
                            elif i==7: # month
                                f_out.write(str(aux_date.month)+'\t')
                            elif i==8: # day
                                f_out.write(str(aux_date.day)+'\t')
                            else:
                                f_out.write(aux[i]+'\t')
                        else:
                            f_out.write(aux[i])
                    f_out.write('\n')
                    #----
            del aux

    #print(len(count_storms))



